In [5]:
# !gsutil -m cp -r gs://neurips-adc-bucket/raw/train/100468857 ../data/raw/train
# !gsutil -m cp -r gs://neurips-adc-bucket/raw/train/1005054328 ../data/raw/train
# !gsutil cp gs://neurips-adc-bucket/raw/* ../data/raw/

In [9]:
from google.cloud import storage
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions, StandardOptions
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner
import apache_beam.runners.interactive.interactive_beam as ibeam
import os
import subprocess
import glob
import shutil

In [2]:
storage_client = storage.Client()

In [3]:
blobs = storage_client.list_blobs(
    bucket_or_name="neurips-adc-bucket", match_glob="raw/**/*", delimiter="/"
)

In [5]:
[blob for blob in blobs.prefixes]

['raw/test/', 'raw/train/']

In [6]:
def list_blobs(bucket_name):
    """Lists all the blobs in the bucket."""
    # bucket_name = "your-bucket-name"

    storage_client = storage.Client()

    # Note: Client.list_blobs requires at least package version 1.17.0.
    blobs = storage_client.list_blobs(bucket_name)

    # Note: The call returns a response only when the iterator is consumed.
    for blob in blobs:
        print(blob.name)

In [17]:
def copy_to(element, dest: str):
    sufix = "/".join(element.split("/")[-2:])
    output = os.path.join(dest, sufix)
    shutil.copytree(src=element, dst=output)

In [26]:
with beam.Pipeline() as pipeline:
    outputs = (
        pipeline
        | "Create initial values" >> beam.Create(glob.glob("../data/raw/**/*"))
        | "Copy the folder" >> beam.Map(lambda x: copy_to(x, "../data/output"))
    )
    outputs | beam.Map(print)

None
None


Next setp appliquer la transformation de bout en bout dans le notebook.
quand ce sera bon on va voir comment intéger cela dans un ExampleGen